In [5]:
import pathlib
import os
import random
import time
import timeit
import functools
from collections import OrderedDict
import requests
import sys

from time import sleep

### Template

#### 2-level decorator

In [ ]:
def profile(f):
    @functools.wraps(f)
    def deco(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        print('Elapsed time', f'({f.__name__}): {time.time() - start}s')
        return result

    return deco


@profile
def foo(): ...

#### 3-level decorator

In [ ]:
def profile(msg='Elapsed time', file=sys.stdout):
    def internal(f):
        @functools.wraps(f)
        def deco(*args, **kwargs):
            start = time.time()
            result = f(*args, **kwargs)
            print(msg, f'({f.__name__}): {time.time() - start}s', file=file)
            return result

        return deco

    return internal


@profile()
def foo(): ...


# Real life examples


In [19]:
def profile(msg='Elapsed time'):
    def internal(f):
        @functools.wraps(f)
        def deco(*args, **kwargs):
            start = time.time()
            result = f(*args, **kwargs)
            print(msg, f'({f.__name__}): {time.time() - start}s')
            return result

        return deco

    return internal

In [20]:
# 0 1 2 3 4 5 6 7 8
# 0 1 1 2 3 5 8 13 ...
def fibo(n):
    """Super inefficient fibo function"""
    if n < 2:
        return n
    else:
        return fibo(n - 1) + fibo(n - 2)


fibo(7)

13

In [25]:
@profile()
def fibo(n):
    """Super inefficient fibo function"""
    if n < 2:
        return n
    else:
        return fibo(n - 1) + fibo(n - 2)


fibo(10)

Elapsed time (fibo): 9.5367431640625e-07s
Elapsed time (fibo): 2.1457672119140625e-06s
Elapsed time (fibo): 0.00048089027404785156s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0004971027374267578s
Elapsed time (fibo): 9.5367431640625e-07s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 1.4781951904296875e-05s
Elapsed time (fibo): 0.0005292892456054688s
Elapsed time (fibo): 9.5367431640625e-07s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 1.4781951904296875e-05s
Elapsed time (fibo): 1.1920928955078125e-06s
Elapsed time (fibo): 3.0994415283203125e-05s
Elapsed time (fibo): 0.0005750656127929688s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 1.4781951904296875e-05s
Elapsed time (fibo): 7.152557373046875e-07s
Elapsed time (fibo): 3.0040740966796875e-05s
Elapsed time (fibo): 9.5367431640625e-07s
Elapsed time (fibo): 1.1920928955078125e-06s
Elapsed time (fibo): 1.621246337890625e-05s
Elapsed time (fibo): 6.29425048828125e-05s
Elapsed time (fibo): 0.00065398216

55

#### Recursion support

In [2]:
def profile(msg="Elapsed time for function"):
    def internal(f):
        @functools.wraps(f)
        def deco(*args, **kwargs):
            start = time.time()
            deco._num_call += 1
            result = f(*args, **kwargs)
            deco._num_call -= 1

            if deco._num_call == 0:
                print(msg, f'{f.__name__}: {time.time() - start}s')
            return result

        deco._num_call = 0
        return deco

    return internal


@profile()
def fibo(n):
    """Super inefficient fibo function"""
    if n < 2:
        return n
    else:
        return fibo(n - 1) + fibo(n - 2)


fibo(10)
help(fibo)

NameError: name 'functools' is not defined

#### Exponential backoff

In [33]:
def repeate(max_repeat=10):
    def internal(f):
        @functools.wraps(f)
        def repeater(*args, **kwargs):
            while repeater._num_repeats <= max_repeat:
                try:
                    return f(*args, **kwargs)
                except Exception as ex:
                    if repeater._num_repeats == max_repeat:
                        raise
                    else:
                        print(
                            f'Failed after {repeater._num_repeats + 1} times, trying again after {2 ** repeater._num_repeats} sec...')
                        sleep(2 ** repeater._num_repeats)
                        repeater._num_repeats += 1

        repeater._num_repeats = 0
        return repeater

    return internal


@repeate(max_repeat=4)
# @repeate()
# @repeate # note the difference
def connect_to_server(*args):
    print('Trying to connect: ', *args)
    if sum(random.choices([0, 1], [0.8, 0.2])) == 0:
        raise RuntimeError('Failed to connect')
    print('SUCCESS!')


connect_to_server('google.com')

Trying to connect:  google.com
Failed after 1 times, trying again after 1 sec...
Trying to connect:  google.com
Failed after 2 times, trying again after 2 sec...
Trying to connect:  google.com
Failed after 3 times, trying again after 4 sec...
Trying to connect:  google.com
Failed after 4 times, trying again after 8 sec...
Trying to connect:  google.com
SUCCESS!


#### Cache

In [36]:
def cache(f):
    @functools.wraps(f)
    def deco(*args):
        if args in deco._cache:
            return deco._cache[args]

        result = f(*args)

        deco._cache[args] = result

        return result

    deco._cache = {}

    return deco

In [12]:
def cache(max_limit=64):
    def internal(f):
        @functools.wraps(f)
        def deco(*args, **kwargs):
            cache_key = (args, tuple(kwargs.items()))
            if cache_key in deco._cache:
                # переносимо в кінець списку
                deco._cache.move_to_end(cache_key, last=True)
                return deco._cache[cache_key]
                print(deco._cache[cache_key])

            result = f(*args, **kwargs)
            # видаляємо якшо досягли ліміта
            if len(deco._cache) >= max_limit:
                # видаляємо перший елемент
                deco._cache.popitem(last=False)
            deco._cache[cache_key] = result
            return result
            print(result)

        deco._cache = OrderedDict()
        return deco
        print(deco)

    return internal
    print(internal)

In [13]:
# @profile(msg='Elapsed time')
# @cache(max_limit=5)
@profile(msg='Elapsed time')
@cache
def fetch_url(url, first_n=100):
    """Fetch a given url"""
    res = requests.get(url)
    return res.content[:first_n] if first_n else res.content
fetch_url('https://google.com')
fetch_url('https://google.com')
fetch_url('https://google.com')
fetch_url('https://ithillel.ua')
fetch_url('https://dou.ua')
fetch_url('https://ain.ua')
fetch_url('https://youtube.com')
#fetch_url(url='https://reddit.com')
#fetch_url._cache

Elapsed time internal: 1.2159347534179688e-05s
Elapsed time internal: 1.3113021850585938e-05s
Elapsed time internal: 9.059906005859375e-06s
Elapsed time internal: 7.867813110351562e-06s
Elapsed time internal: 7.867813110351562e-06s
Elapsed time internal: 7.867813110351562e-06s
Elapsed time internal: 7.152557373046875e-06s


<function __main__.cache.<locals>.internal.<locals>.deco>